In [5]:
import panel as pn #Biblioteca usada para criar interfaces web interativas e painéis de controle.
import matplotlib.pyplot as plt #Biblioteca para criar gráficos.
import numpy as np #Biblioteca para operações matemáticas, como manipulação de arrays.
from io import BytesIO #Usado para manipulação de fluxos de dados em memória, aqui utilizado para 
                       #armazenar imagens em formato binário.

# Inicializa, carregando suas extensões e preparando o ambiente para a interface gráfica.
pn.extension()

# Função para calcular o valor com base no prêmio, sinistro, taxa e no montante fornecido
def calculate_values(premium_percentage, sinistro_percentage, taxa_percentage, amount):
    try:
        # A função float converte os percentuais para valores e depois é realizado o calculo
        premium_value = float(premium_percentage) * float(amount) / 100
        sinistro_value = float(sinistro_percentage) * float(amount) / 100
        taxa_value = float(taxa_percentage) * float(amount) / 100
        return premium_value, sinistro_value, taxa_value
    except ValueError:
        return None, None, None

# A função calculate_and_plot é utilizada para gerar o calculo definido acima (calculate_values)
# e para gerar o gráfico
#A função calculate_and_plot combina a lógica de cálculo (calculate_values) com a geração de um gráfico visual,
#e gerencia erros que podem surgir devido a entradas inválidas.
def calculate_and_plot(amount, premium_percentage, sinistro_percentage, taxa_percentage):
    try:
        # Converte os percentuais e o montante para valores
        premium_value, sinistro_value, taxa_value = calculate_values(
            premium_percentage, sinistro_percentage, taxa_percentage, amount
        )

        # Verifica se os valores são válidos
        if None in [premium_value, sinistro_value, taxa_value]:
            return (
                "<div style='padding: 15px; border: 1px solid #f44336; border-radius: 5px; background-color: #fbe9e7; color: #f44336;'>"
                "    Por favor, insira valores válidos para o prêmio, sinistro, taxa e o montante."
                "</div>",
                None
            )

        # Calcula CMR (Custo Médio de Risco)
        cmr_value = premium_value + sinistro_value + taxa_value

        # Gera o gráfico
        fig, ax = plt.subplots() #cria uma nova figura chamada de "fig" e um conjunto de eixos "ax"
        indices = np.arange(4)  #Cria um array de 4 índices (0, 1, 2, 3) para o gráfico que são usados no eixo x
        values = [premium_value, sinistro_value, taxa_value, cmr_value] #valores que vão entrar no gráfico
        labels = ['Prêmio', 'Sinistro', 'Taxa', 'CMR'] #os rótulos que vão entrar no gráfico no eixo x
        bar_width = 0.5 #define a largura das barras
        opacity = 0.8 #define a transparência das barras

        # Gráfico com índices no eixo X e valores no eixo Y
        #Essa função chama um gráfico de barras com as definições já feitas acima
        bars = ax.bar(indices, values, bar_width, alpha=opacity, color=['b', 'r', 'g', 'purple'], label=labels)

        # Configura o gráfico
        ax.set_xlabel('Categorias') #Define o rótulo do eixo X como "Categorias"
        ax.set_ylabel('Valor Calculado') #Define o rótulo do eixo Y como "Valor Calculado"
        ax.set_title('Valores Calculados a partir do Montante e Percentuais') #Define o título do gráfico
        ax.set_xticks(indices) #Define as posições das marcas no eixo X que corresponde aos indices criados acima
                        #ou seja, uma barra fica posicionada na posição 0, a outra na 1 e assim sucessivamente
        ax.set_xticklabels(labels) #Define os rótulos para as marcas do eixo X usando a variável labels (nomes das variáveis)
        ax.legend() #Adiciona uma legenda ao gráfico, usando os rótulos fornecidos na criação das barras.

        # Salva o gráfico em um buffer de imagem
        buf = BytesIO() #cria um buffer de memória em bytes para armazenar o gráfico temporariamente antes de
                        #ele ser exibido
        plt.savefig(buf, format='png') #salva a figura gerada acima dentro da variável buf, em formato png,
        #em vez de salvar diretamente no disco. Isso permite que a imagem seja manipulada ou exibida imediatamente.
        buf.seek(0) #permite que a leitura do gráfico guardado em buf seja feita sempre do início. sempre que ele
                    #alterar, o gráfico vai ser lido desde o início.
        plt.close(fig) #fecha a figura e libera a imagem dela.

        # Converte a imagem em um objeto Image para o Panel e define diversas características do gráfico
        return (
            "<div style='padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9;'>"
            f"<p>Montante: <span style='color: #007bff;'>{amount}</span></p>"
            f"<p>Prêmio (%): <span style='color: #007bff;'>{premium_percentage}%</span></p>"
            f"<p>Sinistro (%): <span style='color: #007bff;'>{sinistro_percentage}%</span></p>"
            f"<p>Taxa (%): <span style='color: #007bff;'>{taxa_percentage}%</span></p>"
            f"<p><strong>Valor Prêmio:</strong> <span style='color: #28a745;'>{premium_value}</span></p>"
            f"<p><strong>Valor Sinistro:</strong> <span style='color: #dc3545;'>{sinistro_value}</span></p>"
            f"<p><strong>Valor Taxa:</strong> <span style='color: #17a2b8;'>{taxa_value}</span></p>"
            f"<p><strong>CMR (Custo Médio de Risco):</strong> <span style='color: #6f42c1;'>{cmr_value}</span></p>"
            "</div>",
            pn.pane.Matplotlib(fig, format='png', width=600, height=400)
        )
      # Tratamento de algum erro que possa vir a ocorrer na execução, indicando que nenhum gráfico foi gerado  
    except Exception as e:
        return (
            "<h3 style='color: #333;'>Erro:</h3>"
            "<div style='padding: 15px; border: 1px solid #f44336; border-radius: 5px; background-color: #fbe9e7; color: #f44336;'>"
            f"    {str(e)}"
            "</div>",
            None
        )
        
# Criação dos Widgets: botões iterativos, campos de textos, entre outros
amount_input = pn.widgets.TextInput(value='1000', name='Montante', width=100) #local que entra o montante
premium_input = pn.widgets.TextInput(value='20', name='Prêmio (%)', width=100) #local que entra o prêmio
sinistro_input = pn.widgets.TextInput(value='10', name='Sinistro (%)', width=100) #local que entra o sinistro
taxa_input = pn.widgets.TextInput(value='5', name='Taxa (%)', width=100) #local que entra a taxa
button = pn.widgets.Button(name='Calcular') #botão calcular

# Organiza os widgets de entrada em uma coluna (sizing_mode define o tamanho da sua caixinha para ocupar o todo, neste caso)
inputs_column = pn.Column(amount_input, premium_input, sinistro_input, taxa_input, sizing_mode='stretch_width')

# Função utilizada para atualizar o resultado
@pn.depends(button.param.clicks, amount_input.param.value, premium_input.param.value, sinistro_input.param.value, taxa_input.param.value)
def update_result(clicks, amount, premium_percentage, sinistro_percentage, taxa_percentage):
    # Converte os valores para strings e limpa entradas vazias, se for vazio ele coloca 0
    premium_percentage_str = premium_percentage if premium_percentage else '0'
    sinistro_percentage_str = sinistro_percentage if sinistro_percentage else '0'
    taxa_percentage_str = taxa_percentage if taxa_percentage else '0'
    #as funções result e plot realiza o cálculo e gera o gráfico respectivamente e o return pn.row arruma o
    #resultado e o gráfico lado a lado
    result, plot = calculate_and_plot(amount, premium_percentage_str, sinistro_percentage_str, taxa_percentage_str)
    return pn.Row(
        pn.pane.HTML(result, width=400),
        plot
    )

# Caminho para a imagem da empresa (use o caminho local ou URL)
company_logo_path = 'C:/Users/jareias/Documents/Junto/Imagem/logo.jpg'  # Altere para o caminho correto da sua imagem

# Carregar a imagem da empresa
company_logo = pn.pane.Image(company_logo_path, width=200, height=100)

# Layout do painel - Ordem dos objetos que serão mostrados
layout = pn.Column(
    company_logo,  # Adiciona a imagem da empresa no topo
    pn.pane.Markdown("# Calculadora de Valores a partir do Montante e Percentuais"),
    inputs_column,
    button,
    update_result,
    sizing_mode='stretch_width'
)

# Mostrar o painel
layout.show()

Launching server at http://localhost:63466


In [26]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     -- ------------------------------------- 10.2/165.9 kB ? eta -:--:--
     -- ------------------------------------- 10.2/165.9 kB ? eta -:--:--
     --------- --------------------------- 41.0/165.9 kB 487.6 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/165.9 kB 651.6 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/165.9 kB 651.6 kB/s eta 0:00:01
     ------------------------------------ 165.9/165.9 kB 662.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/7.8 MB 812.7 kB/s eta 0:00:10
   ------------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install matplotlib.pyplot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install jupyter_bokeh

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/148.6 kB ? eta -:--:--
   -------- ------------------------------- 30.7/148.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 148.6/148.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ---------------------------------------- 139.4/139.4 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   ---------------------------------------- 214.4/214.4 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.3 MB 20.9 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.3 MB 19.6 MB/s eta 0:00:01
   ---------------------------------------  2.3/2.3 MB 21.3 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 14.9 MB/s eta 0:00:00
Note: you may


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install panel

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.7 MB ? eta -:--:--
   - -------------------------------------- 0.7/24.7 MB 22.1 MB/s eta 0:00:02
   --- ------------------------------------ 1.9/24.7 MB 20.0 MB/s eta 0:00:02
   ---- ----------------------------------- 2.9/24.7 MB 20.8 MB/s eta 0:00:02
   ------ --------------------------------- 4.2/24.7 MB 22.1 MB/s eta 0:00:01
   -------- ------------------------------- 5.4/24.7 MB 21.4 MB/s eta 0:00:01
   ---------- ----------------------------- 6.2/24.7 MB 22.1 MB/s eta 0:00:01
   ----------- ---------------------------- 7.3/24.7 MB 21.1 MB/s eta 0:00:01
  


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install dash

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.5 MB 1.3 MB/s eta 0:00:06
   ---------------------------------------- 0.1/7.5 MB 1.1 MB/s eta 0:00:07
   ---------------------------------------- 0.1/7.5 MB 491.5 kB/s eta 0:00:16
    --------------------------------------- 0.2/7.5 MB 833.5 kB/s eta 0:00:09
   - -------------------------------------- 0.2/7.5 MB 908.0 kB/s eta 0:00:09
   - -------------------------------------- 0.2/7.5 MB 808.4 kB/s eta 0:00:09
   - -------------------------------------- 0.3/7.5 MB 905.4 kB/s eta 0:00:08
   - -------------------------------------- 0.3/7.5 MB 936.6 kB/s eta 0:00:08
   - -------------------------------------- 0.4/7.5 MB 857.5 kB/s eta 0:00:09
   -- ------------------------------------- 0.5/7.5 MB 1.1 MB/s eta 0:00:07
   --- ------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import panel as pn
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
from datetime import datetime

# Inicializa o Panel
pn.extension()

# Função para calcular a idade a partir do ano de nascimento
def calculate_age(year):
    try:
        today = datetime.today()
        age = today.year - int(year)
        return age
    except ValueError:
        return None

# Função para realizar o cálculo com cinco anos de nascimento e gerar o gráfico
def calculate_and_plot(*years):
    try:
        # Converte os anos de nascimento em idades
        ages = [calculate_age(year) for year in years]

        # Verifica se todos os anos são válidos
        if None in ages:
            return (
                "<div style='padding: 15px; border: 1px solid #f44336; border-radius: 5px; background-color: #fbe9e7; color: #f44336;'>"
                "    Por favor, insira anos válidos."
                "</div>",
                None
            )

        # Gera o gráfico
        fig, ax = plt.subplots()
        indices = np.arange(1, 6)  # Índices dos anos para o eixo X
        bar_width = 0.5
        opacity = 0.8

        # Gráfico com anos no eixo X e idades no eixo Y
        bars = ax.bar(indices, ages, bar_width, alpha=opacity, color='b', label='Idades')

        # Configura o gráfico
        ax.set_xlabel('Ano de Nascimento')
        ax.set_ylabel('Idade')
        ax.set_title('Idades Calculadas a partir dos Anos de Nascimento')
        ax.set_xticks(indices)
        ax.set_xticklabels(years)
        ax.legend()

        # Salva o gráfico em um buffer de imagem
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.close(fig)

        # Converte a imagem em um objeto Image para o Panel
        return (
            "<h3 style='color: #333;'>Resultados:</h3>"
            "<div style='padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9;'>"
            + "".join([f"<p>Ano de Nascimento {i+1}: <span style='color: #007bff;'>{years[i]}</span></p>"
                       f"<p><strong>Idade:</strong> <span style='color: #28a745;'>{ages[i]}</span></p><hr>"
                       for i in range(5)]) +
            "</div>",
            pn.pane.Matplotlib(fig, format='png', width=600, height=400)
        )
        
    except Exception as e:
        return (
            "<h3 style='color: #333;'>Erro:</h3>"
            "<div style='padding: 15px; border: 1px solid #f44336; border-radius: 5px; background-color: #fbe9e7; color: #f44336;'>"
            f"    {str(e)}"
            "</div>",
            None
        )

# Widgets
year_inputs = [pn.widgets.TextInput(value='2000', name=f'Ano de Nascimento {i+1}', width=100) for i in range(5)]
button = pn.widgets.Button(name='Calcular')

# Organiza os widgets de entrada em uma linha
year_inputs_row = pn.Row(*year_inputs)

# Função para atualizar o resultado
@pn.depends(button.param.clicks, *[year_input.param.value for year_input in year_inputs])
def update_result(clicks, *year_values):
    # Limpa entradas vazias e converte anos para strings
    year_values_str = [year for year in year_values if year]
    result, plot = calculate_and_plot(*year_values_str)
    return pn.Row(
        pn.pane.HTML(result, width=400),
        plot
    )

# Caminho para a imagem da empresa (use o caminho local ou URL)
company_logo_path = 'C:/Users/jareias/Documents/Junto/Imagem/logo.jpg'  # Altere para o caminho correto da sua imagem

# Carregar a imagem da empresa
company_logo = pn.pane.Image(company_logo_path, width=200, height=100)

# Layout do painel
layout = pn.Column(
    company_logo,  # Adiciona a imagem da empresa no topo
    pn.pane.Markdown("# Calculadora de Idades a partir dos Anos de Nascimento"),
    year_inputs_row,
    button,
    update_result,
    sizing_mode='stretch_width'
)

# Mostrar o painel
layout.show()

Launching server at http://localhost:52023
